In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

!ls

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:15 http

In [5]:
#import spark.sql.functions as sqlfunc
df = spark.read.csv("challenge.csv", header = True)
df.show()

+---------------+--------------+-----------------+----------+
|     ip_address|       Country|      Domain Name|Bytes_used|
+---------------+--------------+-----------------+----------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|
| 119.239.207.13|         China|         youtu.be|        51|
|  68.69.217.210|         China|        adobe.com|        10|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|
|   211.13.10.68|     Indonesia|          hud.gov|        29|
|   239.80.21.97|      Suriname|       smh.com.au|       218|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|
| 127.242.24.138|         China| surveymonkey.com|       123|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|
|   237.54.11.63|         China|       amazon.com|        83|
| 252.141.157.25|         Japan|      cornell.edu|       374|
|185.220.128.248|       Belgium|       weebly.com|       389|
|   151.77.19.45|   Afghanistan|independent.co.uk|       282|
|  9.161

In [8]:
from pyspark.sql.functions import *
df1 = df.withColumn("is_Mexico", when(df.Country == 'Mexico', 'Yes').otherwise('No'))
df1.show()

+---------------+--------------+-----------------+----------+---------+
|     ip_address|       Country|      Domain Name|Bytes_used|is_Mexico|
+---------------+--------------+-----------------+----------+---------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|       No|
| 119.239.207.13|         China|         youtu.be|        51|       No|
|  68.69.217.210|         China|        adobe.com|        10|       No|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|       No|
|   211.13.10.68|     Indonesia|          hud.gov|        29|       No|
|   239.80.21.97|      Suriname|       smh.com.au|       218|       No|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|       No|
| 127.242.24.138|         China| surveymonkey.com|       123|       No|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|       No|
|   237.54.11.63|         China|       amazon.com|        83|       No|
| 252.141.157.25|         Japan|      cornell.edu|       374|   

In [11]:
df2 = df1.withColumn("int_Bytes", df.Bytes_used.cast('int'))
df2 = df2.groupby('is_Mexico').agg(sum('int_Bytes'))
df2.show()

+---------+--------------+
|is_Mexico|sum(int_Bytes)|
+---------+--------------+
|       No|        508076|
|      Yes|          6293|
+---------+--------------+



In [14]:
df3 = df1.groupby('Country').agg(countDistinct('ip_address').alias('num_ip'))
df3.sort(col('num_ip').desc()).show()

+--------------+------+
|       Country|num_ip|
+--------------+------+
|         China|   172|
|     Indonesia|   114|
|   Philippines|    65|
|        Russia|    56|
|        Brazil|    35|
|        Poland|    31|
|        Sweden|    28|
|         Japan|    25|
|      Portugal|    23|
|Czech Republic|    23|
|        France|    21|
|          Peru|    19|
|      Colombia|    17|
| United States|    15|
|       Ukraine|    14|
|     Argentina|    14|
|        Mexico|    13|
|      Thailand|    12|
|        Canada|    11|
|       Nigeria|    11|
+--------------+------+
only showing top 20 rows

